<a href="https://colab.research.google.com/github/geeneelair/-09-313/blob/main/%D0%94%D0%97_%D0%A2%D0%B5%D1%81%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D1%89%D0%B8%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
from datetime import datetime
from dataclasses import dataclass


@dataclass
class Question:
    """
    Класс Question

    Хранит данные про вопросы
    """
    id: int
    text: str
    choices: list[str]
    correct_answer_idx: int


@dataclass
class QuizRes:
    """
    Класс QuizRes

    Хранит данные о начале теста, о его окончании, количество вопросов и количество правильных ответов

    Методы:
    score_percent - считает процент правильных ответов
    """
    start_time: datetime
    end_time: datetime
    total_answers: int
    correct_answers: int

    @property
    def score_percent(self):
        return self.correct_answers / self.total_answers * 100


class QuestionFileFormatError(Exception):
    pass


def load_questions(path) -> list[Question]:
    """
    Функция load_questions

    Загружает вопросы из файла по пути path
    """
    if not os.path.exists(path):
        raise FileNotFoundError
    questions = []
    with open(path, encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('|')
            if len(parts) != 4:
                raise QuestionFileFormatError('Неверный формат вопроса')
            question_id = int(parts[0])
            question = parts[1]
            answers = parts[2].split(';')
            correct_id = int(parts[3])
            questions.append(Question(question_id, question, answers, correct_id))
    return questions


class QuizEngine:
    """
    Класс QuizEngine

    Движок викторины, запускает тест и обрабатывает ответы пользователя

    Методы:
    run_quiz - запуск теста
    _get_user_answer - получить из консоли ответ пользователя
    """
    def __init__(self, questions: list[Question]):
        self.questions = questions.copy()
        random.shuffle(self.questions)
        self.correct_answers: int = 0
        self.start_time: datetime | None = None
        self.end_time: datetime | None = None

    def run_quiz(self):
        self.start_time = datetime.now()

        for idx, question in enumerate(self.questions, 1):
            print(f'\nВопрос {idx}/{len(self.questions)}: {question.text}')
            for i, answer in enumerate(question.choices, 1):
                print(f'{i}. {answer}')
            user_answer = self._get_user_answer()
            if user_answer == question.correct_answer_idx + 1:
                print('> Абсолютно верно')
                self.correct_answers += 1
            else:
                print(f'> Неправильно, правильный ответ - {question.correct_answer_idx + 1}')
        self.end_time = datetime.now()

        return QuizRes(start_time=self.start_time,
                       end_time=self.end_time,
                       total_answers=len(self.questions),
                       correct_answers=self.correct_answers)

    @staticmethod
    def _get_user_answer():
        while True:
            ans = int(input('Введите номер ответа: ').strip())
            if 1 <= ans <= 4:
                return ans
            print(f"Пожалуйста, введите целое число от 1 до 4.")


def save_results(result: QuizRes, path: str = 'results.txt'):
    """
    Функция save_results

    Сохраняет результаты теста в файл
    """
    with open(path, 'a', encoding='utf-8') as f:
        f.write(f'Время начала теста: {result.start_time.strftime("%d-%m-%Y %H:%M:%S")}\n')
        f.write(f'Время окончания теста: {result.end_time.strftime("%d-%m-%Y %H:%M:%S")}\n')
        f.write(f"Всего вопросов: {result.total_answers}\n")
        f.write(f"Правильных ответов: {result.correct_answers}\n")
        f.write(f"Процент правильных: {result.score_percent:.2f}%\n")
        f.write("-" * 40 + "\n")


def main():
    file_path = "questions.txt"
    try:
        questions = load_questions(file_path)
    except Exception as e:
        print(f"Ошибка при загрузке: {e}")
        return

    engine = QuizEngine(questions)
    result = engine.run_quiz()
    save_results(result)
    print(f"\nРезультат: {result.correct_answers}/{result.total_answers} "
          f"({result.score_percent:.2f}%)")


if __name__ == "__main__":
    main()
